In [3]:
from config import DATABASE_URI
import psycopg2
from PIL import Image
import io
import numpy as np
import random

In [4]:
def get_images_from_db():
    # Connect to the database
    conn = psycopg2.connect(DATABASE_URI)

    cursor = conn.cursor()
    
    # Query to get all fruit images
    cursor.execute("SELECT fruit_name, image FROM fruits")
    rows = cursor.fetchall()
    
    # Shuffle and take only half
    random.shuffle(rows)
    rows = rows[:len(rows)//2]
    
    images = []
    labels = []
    for row in rows:
        fruit_name, binary_data = row
        image = Image.open(io.BytesIO(binary_data))
        image = image.resize((128, 128))  # Resize for consistency
        images.append(np.array(image))
        labels.append(fruit_name)
    
    cursor.close()
    conn.close()
    
    return np.array(images), np.array(labels)

images, labels = get_images_from_db()
print(f'Retrieved {len(images)} images from the database.')

Retrieved 21922 images from the database.


In [5]:
# Normalize the images
images = images / 255.0

# Encode labels as integers
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [5]:

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [6]:
# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

c:\Users\Javier Alarcon\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

Epoch 1/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 81s 133ms/step - accuracy: 0.5337 - loss: 1.4260 - val_accuracy: 0.8919 - val_loss: 0.3470
Epoch 2/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 64s 116ms/step - accuracy: 0.9115 - loss: 0.2647 - val_accuracy: 0.6940 - val_loss: 1.3460
Epoch 3/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 63s 114ms/step - accuracy: 0.9222 - loss: 0.2493 - val_accuracy: 0.9143 - val_loss: 0.2744
Epoch 4/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 63s 114ms/step - accuracy: 0.9758 - loss: 0.0729 - val_accuracy: 0.8800 - val_loss: 0.4934
Epoch 5/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 62s 113ms/step - accuracy: 0.9635 - loss: 0.1194 - val_accuracy: 0.9202 - val_loss: 0.3031
Epoch 6/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 62s 113ms/step - accuracy: 0.9882 - loss: 0.0365 - val_accuracy: 0.9209 - val_loss: 0.3384
Epoch 7/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 63s 114ms/step - accuracy: 0.9911 - loss: 0.0287 - val_accuracy: 0.9152 - val_loss: 0.3506
Epoch 8/10
549/549 ━━━━━━━━━━━━━━━━━━━━ 62s 114ms/step - accuracy: 0.9932 - loss: 0

In [8]:
# Save the model
model.save('fruit_classifier_model.h5')

In [9]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

138/138 - 5s - 35ms/step - accuracy: 0.9147 - loss: 0.4433
Test accuracy: 0.914709210395813


In [11]:
# Predict on test data
predictions = model.predict(X_test)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

138/138 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step


In [13]:
# Print some of the predictions to see how the model performs
for i in range(50):
    print(f'Actual: {label_encoder.inverse_transform([y_test[i]])[0]}, Predicted: {predicted_labels[i]}')

Actual: Tomatoes, Predicted: Tomatoes
Actual: Guava, Predicted: Guava
Actual: Pear, Predicted: Pear
Actual: Muskmelon, Predicted: Muskmelon
Actual: Persimmon, Predicted: Persimmon
Actual: Guava, Predicted: Guava
Actual: Pear, Predicted: Pear
Actual: Pear, Predicted: Pear
Actual: Guava, Predicted: Guava
Actual: Persimmon, Predicted: Persimmon
Actual: Peach, Predicted: Peach
Actual: Guava, Predicted: Guava
Actual: Mango, Predicted: Mango
Actual: Pear, Predicted: Pear
Actual: Tomatoes, Predicted: Tomatoes
Actual: Banana, Predicted: Banana
Actual: Pitaya, Predicted: Pitaya
Actual: Muskmelon, Predicted: Guava
Actual: Guava, Predicted: Guava
Actual: Kiwi, Predicted: Kiwi
Actual: Apple, Predicted: Apple
Actual: Pear, Predicted: Pear
Actual: Kiwi, Predicted: Guava
Actual: Pitaya, Predicted: Pitaya
Actual: Banana, Predicted: Banana
Actual: Peach, Predicted: Peach
Actual: Orange, Predicted: Orange
Actual: Guava, Predicted: Guava
Actual: Mango, Predicted: Mango
Actual: Kiwi, Predicted: Pomegranat

In [6]:
# Save the label encoder
import pickle

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)